In [1]:
from dotenv import load_dotenv
import os
from langchain_neo4j import Neo4jGraph

from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from pydantic import BaseModel, Field
# from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer

from langchain_neo4j import Neo4jVector
# from langchain_openai import OpenAIEmbeddings
from langchain_neo4j.vectorstores.neo4j_vector import remove_lucene_chars

c:\Users\Subrata Samanta\miniconda3\envs\genai\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [41]:
from langchain_groq import ChatGroq

load_dotenv()
chat = ChatGroq(temperature=0, model_name="deepseek-r1-distill-llama-70b")





print(chat.invoke("Write a poem about time travel."))

content='<think>\nOkay, so I need to write a poem about time travel. Hmm, where do I start? Time travel is such a vast and imaginative topic. I guess I should think about the different aspects of time travel that could be poetic. Maybe the concept of moving through time, the emotions involved, the possibilities, and the consequences.\n\nFirst, I should consider the structure of the poem. Do I want it to be free verse or follow a specific rhyme scheme? Rhyming might give it a more traditional feel, which could be nice. Maybe something with a consistent meter to give it a rhythmic flow, kind of like the passage of time itself.\n\nImagery is important in poetry, so I should think about what images evoke time travel. Clocks, hourglasses, maybe a time machine. But I don\'t want it to be too cliché. Perhaps more abstract imagery, like shadows, echoes, or the flow of a river, which can symbolize time.\n\nEmotions are key too. Time travel can bring up feelings of nostalgia, wonder, maybe even 

In [29]:
from euriai import EuriaiLangChainLLM

chat = EuriaiLangChainLLM(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4.1-nano",
    temperature=0.7,
    max_tokens=30000
)
print(chat.invoke("Write a poem about time travel."))

Through veils of seconds, centuries unwind,  
A whispered thread in the fabric of mind.  
Steps across eras, both near and far,  
A dance with shadows of what we are.

Clocks that tick in cosmic rhyme,  
Unraveling the secrets of time.  
Forward to futures yet unseen,  
Backward to moments long between.

A leap through ages, swift and bright,  
Chasing dawns into endless night.  
History’s echo, a fleeting ghost,  
In the traveler’s heart, a longing host.

Bound by neither chain nor gate,  
Time’s river bends, it’s never too late.  
To glimpse the past, or future’s glow,  
A voyage where all things flow.


In [45]:

AURA_INSTANCENAME = os.environ["AURA_INSTANCENAME"]
NEO4J_URI = os.environ["NEO4J_URI"]
NEO4J_USERNAME = os.environ["NEO4J_USERNAME"]
NEO4J_PASSWORD = os.environ["NEO4J_PASSWORD"]
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# OPENAI_ENDPOINT = os.getenv("OPENAI_ENDPOINT")

In [46]:
kg = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
) #database=NEO4J_DATABASE,


In [17]:
# # read the wikipedia page for the Roman Empire
from langchain_community.document_loaders import PyPDFLoader

raw_documents = PyPDFLoader("NASDAQ_AAPL_2024.pdf").load()



In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# # # # Define chunking strategy
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=200)
documents = text_splitter.split_documents(raw_documents)
print(len(documents))


# graph_documents = llm_transformer.convert_to_graph_documents(documents)

448


In [55]:
for doc in documents[:10]:
    print(doc.page_content)
    print("==="*50)

UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☒  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended September 28, 2024
or
☐  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from              to             .
Commission File Number: 001-36743
Apple Inc.
(Exact name of Registrant as specified in its charter)
California 94-2404110
(State or other jurisdictionof incorporation or organization) (I.R.S. Employer Identification No.)
One Apple Park Way
Cupertino, California 95014
(Address of principal executive offices) (Zip Code)
(408) 996-1010
(Registrant’s telephone number, including area code)
Securities registered pursuant to Section 12(b) of the Act:
Title of each class Trading symbol(s) Name of each exchange on which registered
Common Stock, $0.00001 par value per shareAAPL The Nasdaq Stock Market LLC
0.000% Notes du

In [43]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

llm_transformer = LLMGraphTransformer(llm=chat)

# Function to process a single document
def process_document(doc):
    return llm_transformer.convert_to_graph_documents([doc])


graph_documents = []
for doc in tqdm(documents[:10], desc="Processing Documents"):
    result = process_document(doc)
    graph_documents.extend(result)

# # Use ThreadPoolExecutor to process documents in parallel with progress tracking
# graph_documents = []
# with ThreadPoolExecutor() as executor:
#     with tqdm(total=len(documents), desc="Processing Documents") as pbar:
#         for i in range(0, len(documents), 1):  # Batch size of 10
#             batch = documents[i:i+10]
#             futures = {executor.submit(process_document, doc): doc for doc in batch}
#             for future in futures:
#                 result = future.result()
#                 graph_documents.extend(result)
#                 pbar.update(1)

# # Flatten the list of results
# graph_documents = [item for sublist in graph_documents for item in sublist]

Processing Documents: 100%|██████████| 10/10 [03:37<00:00, 21.72s/it]


In [47]:
graph_documents

[GraphDocument(nodes=[Node(id='Form 10-K', type='Document', properties={}), Node(id='Apple Inc.', type='Company', properties={}), Node(id='One Apple Park Way, Cupertino, California 95014', type='Address', properties={}), Node(id='California', type='State', properties={}), Node(id='94-2404110', type='Ein', properties={}), Node(id='(408) 996-1010', type='Phone number', properties={}), Node(id='Common Stock, $0.00001 Par Value Per Share', type='Security', properties={}), Node(id='Aapl', type='Ticker symbol', properties={}), Node(id='The Nasdaq Stock Market Llc', type='Exchange', properties={}), Node(id='0.000% Notes Due 2025', type='Bond', properties={}), Node(id='0.875% Notes Due 2025', type='Bond', properties={}), Node(id='1.625% Notes Due 2026', type='Bond', properties={}), Node(id='2.000% Notes Due 2027', type='Bond', properties={})], relationships=[Relationship(source=Node(id='Apple Inc.', type='Company', properties={}), target=Node(id='Form 10-K', type='Document', properties={}), ty

In [48]:
# # store to neo4j
res = kg.add_graph_documents(
    graph_documents,
    include_source=True,
    baseEntityLabel=True,
)


In [49]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [56]:
# create vector index
vector_index = Neo4jVector.from_existing_graph(
    hf,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
)


In [57]:
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


In [58]:
chat

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A90C52CE90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A90C3E4450>, model_name='deepseek-r1-distill-llama-70b', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [59]:
entity_chain = prompt | chat.with_structured_output(Entities)

In [60]:
# # Test it out:
res = entity_chain.invoke(
    {"question": "tell me the growth rate of apple in 2024"}
).names
print(res)

['apple']


In [61]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [62]:
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        print(f" Getting Entity: {entity}")
        response = kg.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        # print(response)
        result += "\n".join([el["output"] for el in response])
    return result

In [64]:
print(structured_retriever("tell me the growth rate of apple in 2024"))


 Getting Entity: tell me the growth rate of apple in 2024


ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.fulltext.queryNodes`: Caused by: java.lang.IllegalArgumentException: There is no such fulltext schema index: entity}

In [65]:
# Final retrieval step
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [
        el.page_content for el in vector_index.similarity_search(question)
    ]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    print(f"\nFinal Data::: ==>{final_data}")
    return final_data

In [66]:
# Define the RAG chain
# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)


In [67]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer


_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | chat
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x: x["question"]),
)


In [68]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | chat
    | StrOutputParser()
)

In [69]:
res_hist = chain.invoke(
    {
        "question": "tell me the growth rate of apple in 2024?",
        "chat_history": [

        ],
    }
)

print(f"\n === {res_hist}\n\n")

Search query: tell me the growth rate of apple in 2024?
 Getting Entity: apple


ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.fulltext.queryNodes`: Caused by: java.lang.IllegalArgumentException: There is no such fulltext schema index: entity}